In [39]:
#!pip install optuna

In [41]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [42]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [43]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [46]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy and for loss it will use the minimize
#Sampler suggest the next value based on the past data
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2026-02-20 06:45:56,756] A new study created in memory with name: no-name-402720a5-eb50-4e3e-bb23-765774a04fc5
[I 2026-02-20 06:45:57,276] Trial 0 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 101, 'max_depth': 3}. Best is trial 0 with value: 0.7560521415270017.
[I 2026-02-20 06:45:57,707] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 75, 'max_depth': 15}. Best is trial 1 with value: 0.7672253258845437.
[I 2026-02-20 06:45:58,466] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 134, 'max_depth': 17}. Best is trial 2 with value: 0.7746741154562384.
[I 2026-02-20 06:45:59,249] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 138, 'max_depth': 14}. Best is trial 2 with value: 0.7746741154562384.
[I 2026-02-20 06:45:59,798] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 96, 'max_depth': 18}. Best is trial 2 with value: 0.7746741

In [47]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 184, 'max_depth': 16}


In [48]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [50]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler()) #RandomSearch
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2026-02-20 06:49:50,398] A new study created in memory with name: no-name-95409067-b530-4bf1-9c15-88468a340fbf
[I 2026-02-20 06:49:52,097] Trial 0 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 111, 'max_depth': 3}. Best is trial 0 with value: 0.7597765363128491.
[I 2026-02-20 06:49:54,799] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 173, 'max_depth': 18}. Best is trial 1 with value: 0.7672253258845437.
[I 2026-02-20 06:49:55,708] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 99, 'max_depth': 4}. Best is trial 1 with value: 0.7672253258845437.
[I 2026-02-20 06:49:57,556] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 155, 'max_depth': 6}. Best is trial 1 with value: 0.7672253258845437.
[I 2026-02-20 06:49:58,385] Trial 4 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 67, 'max_depth': 6}. Best is trial 1 with value: 0.7672253258

In [51]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420857
Best hyperparameters: {'n_estimators': 69, 'max_depth': 20}


In [52]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [54]:
# For the GridSearch CV
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [55]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2026-02-20 06:52:26,720] A new study created in memory with name: no-name-8a88bcbe-8cfc-49f8-a565-b5d669d725d0
[I 2026-02-20 06:52:27,595] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-20 06:52:28,826] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-20 06:52:29,113] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-20 06:52:29,690] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-20 06:52:30,265] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [56]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [58]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [59]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [60]:
# 1. Optimization History
plot_optimization_history(study).show()

In [61]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [62]:
# 3. Slice Plot
plot_slice(study).show()

In [63]:
# 4. Contour Plot
plot_contour(study).show()

In [64]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [65]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [66]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [67]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2026-02-20 07:01:11,343] A new study created in memory with name: no-name-cde0884f-fc76-4cc2-99ea-eb73ad5b443e
[I 2026-02-20 07:01:12,039] Trial 0 finished with value: 0.7765363128491621 and parameters: {'classifier': 'RandomForest', 'n_estimators': 167, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 0 with value: 0.7765363128491621.
[I 2026-02-20 07:01:12,069] Trial 1 finished with value: 0.7169459962756052 and parameters: {'classifier': 'SVM', 'C': 0.2178961431049863, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7765363128491621.
[I 2026-02-20 07:01:12,641] Trial 2 finished with value: 0.7616387337057727 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 85, 'learning_rate': 0.18266061356792632, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.7765363128491621.
[I 2026-02-20 07:01:12,671] Trial 3 finished with value: 0.7262569832402234 and parameter

In [68]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11076401863260073, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [69]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.776536,2026-02-20 07:01:11.345919,2026-02-20 07:01:12.039436,0 days 00:00:00.693517,NaN,False,RandomForest,NaN,NaN,NaN,5.0,5.0,2.0,167.0,COMPLETE
1,1,0.716946,2026-02-20 07:01:12.041552,2026-02-20 07:01:12.069909,0 days 00:00:00.028357,0.217896,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.761639,2026-02-20 07:01:12.070936,2026-02-20 07:01:12.641367,0 days 00:00:00.570431,NaN,NaN,GradientBoosting,NaN,NaN,0.182661,4.0,5.0,5.0,85.0,COMPLETE
3,3,0.726257,2026-02-20 07:01:12.642575,2026-02-20 07:01:12.671624,0 days 00:00:00.029049,1.676008,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.685289,2026-02-20 07:01:12.672699,2026-02-20 07:01:12.765552,0 days 00:00:00.092853,57.699112,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2026-02-20 07:02:02.303879,2026-02-20 07:02:02.335538,0 days 00:00:00.031659,0.128251,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2026-02-20 07:02:02.336614,2026-02-20 07:02:02.369695,0 days 00:00:00.033081,0.227738,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.787709,2026-02-20 07:02:02.370785,2026-02-20 07:02:02.402962,0 days 00:00:00.032177,0.100206,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2026-02-20 07:02:02.404152,2026-02-20 07:02:02.438099,0 days 00:00:00.033947,0.279227,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [70]:
#Check the which classifiers
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,69
RandomForest,19
GradientBoosting,12


In [71]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.751707
RandomForest,0.770460
SVM,0.775484


In [72]:
# 1. Optimization History
plot_optimization_history(study).show()

In [73]:
# 3. Slice Plot
plot_slice(study).show()

In [74]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
#! pip install optuna-integration[xgboost]

In [78]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2026-02-20 07:22:48,369] A new study created in memory with name: no-name-40718072-d365-47d7-982b-c84db6247f12


[0]	train-mlogloss:1.08875	eval-mlogloss:1.09003
[1]	train-mlogloss:1.07376	eval-mlogloss:1.07432
[2]	train-mlogloss:1.05911	eval-mlogloss:1.05898
[3]	train-mlogloss:1.04409	eval-mlogloss:1.04360
[4]	train-mlogloss:1.03009	eval-mlogloss:1.02902
[5]	train-mlogloss:1.01568	eval-mlogloss:1.01344
[6]	train-mlogloss:1.00199	eval-mlogloss:0.99931
[7]	train-mlogloss:0.98886	eval-mlogloss:0.98541
[8]	train-mlogloss:0.97589	eval-mlogloss:0.97143
[9]	train-mlogloss:0.96296	eval-mlogloss:0.95791
[10]	train-mlogloss:0.95039	eval-mlogloss:0.94503
[11]	train-mlogloss:0.93824	eval-mlogloss:0.93203
[12]	train-mlogloss:0.93054	eval-mlogloss:0.92368
[13]	train-mlogloss:0.91853	eval-mlogloss:0.91066
[14]	train-mlogloss:0.91118	eval-mlogloss:0.90321
[15]	train-mlogloss:0.90051	eval-mlogloss:0.89184
[16]	train-mlogloss:0.89020	eval-mlogloss:0.88083
[17]	train-mlogloss:0.88122	eval-mlogloss:0.87134
[18]	train-mlogloss:0.87120	eval-mlogloss:0.86140
[19]	train-mlogloss:0.86218	eval-mlogloss:0.85180
[20]	train

[I 2026-02-20 07:22:54,513] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.0008872510643650016, 'alpha': 0.007567440169578506, 'eta': 0.011799031764715332, 'gamma': 9.78833580960291e-08, 'max_depth': 9, 'min_child_weight': 8, 'subsample': 0.6837897707337601, 'colsample_bytree': 0.7006522702688043}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.83574	eval-mlogloss:0.81600
[1]	train-mlogloss:0.65035	eval-mlogloss:0.63432
[2]	train-mlogloss:0.51089	eval-mlogloss:0.49483
[3]	train-mlogloss:0.41225	eval-mlogloss:0.39218
[4]	train-mlogloss:0.33719	eval-mlogloss:0.31985
[5]	train-mlogloss:0.27739	eval-mlogloss:0.25324
[6]	train-mlogloss:0.23060	eval-mlogloss:0.20306
[7]	train-mlogloss:0.19527	eval-mlogloss:0.17296
[8]	train-mlogloss:0.17045	eval-mlogloss:0.14428
[9]	train-mlogloss:0.14811	eval-mlogloss:0.11775
[10]	train-mlogloss:0.12953	eval-mlogloss:0.09842
[11]	train-mlogloss:0.11476	eval-mlogloss:0.08389
[12]	train-mlogloss:0.10323	eval-mlogloss:0.07413
[13]	train-mlogloss:0.09346	eval-mlogloss:0.06434
[14]	train-mlogloss:0.08573	eval-mlogloss:0.05593
[15]	train-mlogloss:0.08004	eval-mlogloss:0.04874
[16]	train-mlogloss:0.07554	eval-mlogloss:0.04306
[17]	train-mlogloss:0.07125	eval-mlogloss:0.03810
[18]	train-mlogloss:0.06769	eval-mlogloss:0.03337
[19]	train-mlogloss:0.06350	eval-mlogloss:0.02979
[20]	train

[I 2026-02-20 07:22:57,113] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.045454734437370754, 'alpha': 0.02832061151287188, 'eta': 0.218325429495776, 'gamma': 0.0002627555008317843, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.6388973906853742, 'colsample_bytree': 0.9837029150085375}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.98373	eval-mlogloss:0.99375
[1]	train-mlogloss:0.75232	eval-mlogloss:0.74276


[I 2026-02-20 07:23:20,201] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.02994	eval-mlogloss:1.03054
[1]	train-mlogloss:0.93703	eval-mlogloss:0.93232


[I 2026-02-20 07:23:20,238] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.0008872510643650016, 'alpha': 0.007567440169578506, 'eta': 0.011799031764715332, 'gamma': 9.78833580960291e-08, 'max_depth': 9, 'min_child_weight': 8, 'subsample': 0.6837897707337601, 'colsample_bytree': 0.7006522702688043}
Best accuracy: 1.0


In [80]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()